In [9]:
from datasets import load_dataset, DatasetDict, Dataset
from pylate import evaluation, indexes, models, retrieve
import matplotlib.pyplot as plt
from tqdm import tqdm
from collections import defaultdict

In [2]:
dataset = load_dataset("sentence-transformers/msmarco-bm25", "triplet", split="train")

In [5]:
fullds = DatasetDict({
    "train": dataset.select(range(500000)),
    "test": dataset.select(range(500000, len(dataset))),
})

In [6]:
fullds.save_to_disk("../propercache/data/colbert_training/msmarco_500k")

Saving the dataset (1/1 shards): 100%|██████████| 2931/2931 [00:00<00:00, 97676.03 examples/s]


In [2]:
msmarco = load_dataset("mteb/msmarco")

In [3]:
msmarcocorp = load_dataset("mteb/msmarco", name="corpus")

In [4]:
msmarcoquery = load_dataset("mteb/msmarco", name="queries")

In [21]:
msmarcocorp

DatasetDict({
    corpus: Dataset({
        features: ['_id', 'title', 'text'],
        num_rows: 8841823
    })
})

In [5]:
qidsegt = set(msmarco['dev']['query-id'])
corpset = set(msmarco['dev']['corpus-id'])

In [6]:
qrels = {}
def qrelrows(row):
    if row['_id'] in qidsegt:
        qrels[row['_id']] = row
    return None

docrels = {}
def docrelrows(row):
    if row['_id'] in corpset:
        docrels[row['_id']] = row
    return None

for i in tqdm(range(len(msmarcoquery['queries']))):
    if msmarcoquery['queries'][i]['_id'] in qidsegt:
        qrels[msmarcoquery['queries'][i]['_id']] = msmarcoquery['queries'][i]


100%|██████████| 509962/509962 [00:13<00:00, 36501.27it/s]


In [7]:
evset = []
for r in msmarco['dev']:
    if r['score'] >0:
        evset.append({
            'qid': r['query-id'],
            'corpus-id': r['corpus-id'],
            'question': qrels[r['query-id']]['text'],
            'pos_chunks': [msmarcocorp['corpus'][int(r['corpus-id'])]['text']],
            'score': r['score'],
        })

In [11]:
import random

In [12]:
# make datastore using all pos_chunks and enough negative chunks to fill up to 20000 chunks
target = 20000
pchunks = [r['pos_chunks'][0] for r in evset]
randinds = random.sample(range(len(msmarcocorp['corpus'])), target - len(pchunks))
randinds = msmarcocorp['corpus'].select(randinds)
extrachunks = list(set([r['text'] for r in randinds]))
newds = Dataset.from_list([{'text': c} for c in pchunks + extrachunks])

In [15]:
Dataset.from_list(evset).select(range(500)).save_to_disk("../propercache/data/evalsets/msmarco_20k_docs")

Saving the dataset (1/1 shards): 100%|██████████| 500/500 [00:00<00:00, 55967.33 examples/s]


In [ ]:
# newds.save_to_disk("../propercache/data/datastores/msmarco_20k_docs")

Saving the dataset (1/1 shards): 100%|██████████| 20000/20000 [00:00<00:00, 256413.17 examples/s]


In [14]:
newds[0]

{'text': 'http://en.wikipedia.org/wiki/William_Bradford_(Plymouth_Colony_governor) William Bradford (c.1590 â\x80\x93 1657) was an English Separatist leader in Leiden, Holland and in Plymouth Colony was a signatory to the Mayflower Compact. He served as Plymouth Colony Governor five times covering about thirty years between 1621 and 1657.'}

In [58]:
evset[5]

{'qid': '320792',
 'corpus-id': '7067677',
 'question': 'how much is a cost to run disneyland',
 'pos_chunks': ["Disney's Theme Parks had an operating cost of 571 million dollars divided by their 11 parks and being open 365 days a year, on average their operating cost per day is around $355,000."],
 'score': 1.0}

In [43]:
len(set([r['question'] for r in evset]))

43

In [46]:
len(set(msmarco['test']['query-id']))


43